<a href="https://colab.research.google.com/github/wojiushilr/ML_hw/blob/master/hw4_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using BERT-insight in Colab

### TPU install

In [ ]:
## ONLY TPU needed
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

#### [RUNME] Install Colab TPU compatible PyTorch/TPU wheels and dependencies

In [ ]:
# VERSION = "20200516"  #@param ["1.5" , "20200516", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

###  ENV Confirmation

In [ ]:
!nvcc --version
!pip -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
pip 19.3.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [ ]:
! git clone https://github.com/wojiushilr/BERT-insight.git

Cloning into 'BERT-insight'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 300 (delta 108), reused 293 (delta 104), pack-reused 0
Receiving objects: 100% (300/300), 1.37 MiB | 3.02 MiB/s, done.
Resolving deltas: 100% (108/108), done.


### 下载数据

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/kaggle/hw4_bert/XNLI-1.0.zip" /content
!cp "/content/drive/My Drive/kaggle/hw4_bert/XNLI-MT-1.0.zip" /content

In [ ]:
! unzip XNLI-1.0.zip
! unzip XNLI-MT-1.0.zip

Archive:  XNLI-1.0.zip
   creating: XNLI-1.0/
  inflating: XNLI-1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/XNLI-1.0/
  inflating: __MACOSX/XNLI-1.0/._.DS_Store  
  inflating: XNLI-1.0/xnli.dev.tsv   
  inflating: __MACOSX/XNLI-1.0/._xnli.dev.tsv  
  inflating: XNLI-1.0/xnli.dev.jsonl  
  inflating: XNLI-1.0/README.md      
  inflating: __MACOSX/XNLI-1.0/._README.md  
  inflating: XNLI-1.0/xnli.test.jsonl  
  inflating: XNLI-1.0/xnli.test.tsv  
  inflating: __MACOSX/XNLI-1.0/._xnli.test.tsv  
  inflating: __MACOSX/._XNLI-1.0     
Archive:  XNLI-MT-1.0.zip
   creating: XNLI-MT-1.0/
   creating: XNLI-MT-1.0/multinli/
  inflating: XNLI-MT-1.0/multinli/multinli.train.de.tsv  
   creating: __MACOSX/XNLI-MT-1.0/
   creating: __MACOSX/XNLI-MT-1.0/multinli/
  inflating: __MACOSX/XNLI-MT-1.0/multinli/._multinli.train.de.tsv  
  inflating: XNLI-MT-1.0/multinli/multinli.train.es.tsv  
  inflating: __MACOSX/XNLI-MT-1.0/multinli/._multinli.train.es.tsv  
  inflating: XNLI-MT-1.

In [ ]:
!cp -r /content/XNLI-1.0 /content/BERT-insight/dataset/
!cp -r /content/XNLI-MT-1.0 /content/BERT-insight//dataset/

In [ ]:
! pip install transformers
! pip install sacremoses

     |████████████████████████████████| 778kB 8.2MB/s 
     |████████████████████████████████| 3.0MB 36.0MB/s 
     |████████████████████████████████| 890kB 54.7MB/s 
     |████████████████████████████████| 1.1MB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1341e4ae5dd9f64f8ff07db4d47f7707ef8c704989cc00ffa75e5487956146c2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
! git clone https://www.github.com/nvidia/apex
% cd apex
! python setup.py install

Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7401 (delta 0), reused 0 (delta 0), pack-reused 7400
Receiving objects: 100% (7401/7401), 13.90 MiB | 9.20 MiB/s, done.
Resolving deltas: 100% (4999/4999), done.
/content/apex


torch.__version__  = 1.6.0+cu101


setup.py:67: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")
running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/apex
copying apex/__

In [ ]:
% cd /content/BERT-insight//bert/
! ls

/content/BERT-insight/bert
download_glue_data.py	     run_glue.py  run_xnli_tpu.py
generate-similarity-data.py  runs	  run_xnli_tpu.sh
models.py		     run_xnli.py  similarity_student.py
picture			     run_xnli.sh  transformers


In [ ]:
! ./run_xnli.sh

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/